# A1111 Model Installer — SD 1.5

Downloads common **SD 1.5** assets for Automatic1111 with visible **progress bars**:

- **Checkpoints** → `/workspace/a1111/models/Stable-diffusion/`
- **VAE** → `/workspace/a1111/models/VAE/`
- **ControlNet (SD15)** → `/workspace/a1111/models/ControlNet/`

You can add or remove items in the lists below. Re-running is safe: existing files are skipped.

> **Note:** Some weights require you to accept a license on Hugging Face and/or set `HF_TOKEN` in the environment.

In [ ]:
# If this is the first run, uncomment to ensure deps exist (idempotent)
# !pip install -q tqdm requests huggingface_hub
print("Downloader ready.")

In [ ]:
from pathlib import Path

# Where A1111 looks for data (matches WEBUI_ROOT in your image)
DATA_ROOT = Path("/workspace/a1111")

# Standard A1111 subfolders
CKPT_DIR = DATA_ROOT / "models/Stable-diffusion"
VAE_DIR  = DATA_ROOT / "models/VAE"
CN_DIR   = DATA_ROOT / "models/ControlNet"

for d in (CKPT_DIR, VAE_DIR, CN_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("Data root:", DATA_ROOT)
print("↳ Checkpoints:", CKPT_DIR)
print("↳ VAE:", VAE_DIR)
print("↳ ControlNet (SD15):", CN_DIR)

In [ ]:
# ---- Progress-enabled download helpers ----
import os, shutil, requests, shlex, subprocess, time
from tqdm import tqdm
from huggingface_hub import hf_hub_download

def human(n: int) -> str:
    for unit in ["B","KB","MB","GB"]:
        if n < 1024:
            return f"{n:.1f} {unit}"
        n /= 1024
    return f"{n:.1f} TB"

def download_direct(url: str, dest: Path):
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print("✔ Exists:", dest)
        return dest
    r = requests.get(url, stream=True)
    r.raise_for_status()
    total = int(r.headers.get('Content-Length', 0))
    tmp = dest.with_suffix(dest.suffix + ".part")
    with open(tmp, 'wb') as f, tqdm(total=total, unit='B', unit_scale=True, desc=dest.name, ncols=80) as bar:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk)
                bar.update(len(chunk))
    tmp.replace(dest)
    print("✔ Saved:", dest, f"({human(dest.stat().st_size)})")
    return dest

def download_hf(repo_id: str, filename: str, dest_dir: Path, dest_name: str|None=None, token: str|None=None):
    dest = dest_dir / (dest_name or filename)
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print("✔ Exists:", dest)
        return dest
    print(f"→ HF: {repo_id}/{filename}")
    path = hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        token=token or os.environ.get("HF_TOKEN"),
        local_dir=dest_dir,
        local_dir_use_symlinks=False,
        resume_download=True,
    )
    if Path(path) != dest:
        shutil.copy2(path, dest)
    print("✔ HF downloaded:", dest, f"({human(dest.stat().st_size)})")
    return dest


## Choose what to download

Toggle items by commenting/uncommenting. You can mix **Hugging Face** items and **direct URLs**.

**Tips**
- If a repo requires access, make sure your pod env includes `HF_TOKEN`.
- You can add any public model URLs — they’ll download with progress.

In [ ]:
# --- Checkpoints (SD 1.5) ---
CHECKPOINTS = [
    # Example: official 1.5 (requires license on HF)
    # {"kind": "hf", "repo": "runwayml/stable-diffusion-v1-5", "filename": "v1-5-pruned.safetensors", "dest": CKPT_DIR / "v1-5-pruned.safetensors"},

    # Put your favorite SD1.5 checkpoints here (HF or direct links)
    # {"kind": "direct", "url": "https://your.cdn/model.safetensors", "dest": CKPT_DIR / "your-model.safetensors"},
]

# --- VAE ---
VAE_FILES = [
    # Popular finetuned VAE (public on HF)
    {"kind": "hf", "repo": "stabilityai/sd-vae-ft-mse", "filename": "vae-ft-mse-840000-ema-pruned.safetensors", "dest": VAE_DIR / "vae-ft-mse-840000-ema-pruned.safetensors"},
]

# --- ControlNet (SD 1.5 series) ---
# Using lllyasviel/control_v11p_sd15_* repos; we save with friendly filenames for A1111
CONTROLNET_SD15 = [
    {"repo": "lllyasviel/control_v11p_sd15_canny",    "filename": "diffusion_pytorch_model.safetensors", "save_as": CN_DIR / "control_v11p_sd15_canny.safetensors"},
    {"repo": "lllyasviel/control_v11p_sd15_depth",    "filename": "diffusion_pytorch_model.safetensors", "save_as": CN_DIR / "control_v11p_sd15_depth.safetensors"},
    {"repo": "lllyasviel/control_v11p_sd15_normalbae","filename": "diffusion_pytorch_model.safetensors", "save_as": CN_DIR / "control_v11p_sd15_normalbae.safetensors"},
    {"repo": "lllyasviel/control_v11p_sd15_openpose", "filename": "diffusion_pytorch_model.safetensors", "save_as": CN_DIR / "control_v11p_sd15_openpose.safetensors"},
    {"repo": "lllyasviel/control_v11p_sd15_lineart",  "filename": "diffusion_pytorch_model.safetensors", "save_as": CN_DIR / "control_v11p_sd15_lineart.safetensors"},
    {"repo": "lllyasviel/control_v11p_sd15_softedge", "filename": "diffusion_pytorch_model.safetensors", "save_as": CN_DIR / "control_v11p_sd15_softedge.safetensors"},
]

print(f"Planned: {len(CHECKPOINTS)} checkpoints, {len(VAE_FILES)} VAE, {len(CONTROLNET_SD15)} ControlNet models")

In [ ]:
# ---- Run downloads ----
errors = []

# Checkpoints
for item in CHECKPOINTS:
    try:
        if item.get("kind") == "hf":
            download_hf(item["repo"], item["filename"], item["dest"].parent, dest_name=item["dest"].name)
        elif item.get("kind") == "direct":
            download_direct(item["url"], item["dest"])
    except Exception as e:
        errors.append(("ckpt", str(item), str(e)))

# VAE
for item in VAE_FILES:
    try:
        if item.get("kind") == "hf":
            download_hf(item["repo"], item["filename"], item["dest"].parent, dest_name=item["dest"].name)
        elif item.get("kind") == "direct":
            download_direct(item["url"], item["dest"])
    except Exception as e:
        errors.append(("vae", str(item), str(e)))

# ControlNet (SD15)
for item in CONTROLNET_SD15:
    try:
        download_hf(item["repo"], item["filename"], item["save_as"].parent, dest_name=item["save_as"].name)
    except Exception as e:
        errors.append(("controlnet", str(item), str(e)))

print("\n=== Summary ===")
for p in sorted(list(CKPT_DIR.glob('*')) + list(VAE_DIR.glob('*')) + list(CN_DIR.glob('*'))):
    try:
        print(f"✔ {p.name:45s} {human(p.stat().st_size)}")
    except Exception:
        pass
if errors:
    print("\nErrors:")
    for kind, item, msg in errors:
        print("✖", kind, "→", item, "\n  ", msg)

## (Optional) Restart the WebUI service

If you installed new ControlNet models or VAEs while A1111 was running, you may want to restart it.
This tries a supervised restart first, then a process restart fallback.

In [ ]:
try:
    # Try supervisorctl (matches your container config)
    cmd = "supervisorctl -c /etc/supervisord.conf restart a1111"
    out = subprocess.run(shlex.split(cmd), capture_output=True, text=True, timeout=20)
    if out.returncode == 0:
        print("✔ Restarted via supervisorctl. Wait ~10–15s, then refresh the WebUI tab.")
        print(out.stdout.strip())
    else:
        print("Supervisorctl restart failed; attempting process restart…")
        # Find and kill the running launch.py process; supervisord will respawn it
        subprocess.run(["pkill", "-f", "launch.py"], check=False)
        time.sleep(5)
        print("✔ WebUI process terminated; supervisord will auto-restart it. Refresh in ~10–15s.")
except Exception as e:
    print("Restart attempt encountered an error:", e)
    print("If needed, run this manually in a Terminal:")
    print("supervisorctl -c /etc/supervisord.conf restart a1111")